In [1]:
%config Completer.use_jedi = False
import warnings

warnings.filterwarnings(action='ignore')

# 부스팅(Boosting)
초기에는 모든 데이터 포인트에 동일한 가중치를 할당하고 점차 학습이 진행되면서 올바르게 분류된 데이터 포인트의 가중치는 감소시키는 반면에 잘못 분류된 데이터 포인트의 가중치는 증가시킨다.
결과적으로 학습이 진행되면서 학습는 분류하기 어려운 데이터에 집중하게 되고 이전 단계에서 만들어진 학습기는 다음 단계에서 사용할 트레이닝 셋의 가중치를 반영하는 데 사용하므로 부스팅인 배깅과는 달리 이전 분류기의 영향을 받는다.

배깅에서는 각 데이터 포인트가 추출될 확률이 모두 동일했지만, 부스팅에서는 각 데이터 포인트에 할당된 가중치에 비례해서 추출된다.

# 에이다 부스트(Adaboost)
부스팅은 약한 학습기 여러 개를 모아 하나의 강한 학습기를 만드는 방법으로 개별적으로는 약한 학습 모형이지만, 이와 같은 모형을 다수 생성하고 부스팅을 적용함으로써 강한 학습기가 만들어진다.

보팅이나 배깅은 모델이 병려적으로 수행된다. 10개의 모델이 이다면 10개의 모델을 동시에 학습시킬 수 있다는 뜻이다. 이에 반해 부스팅은 여러 약한 학습기가 순차적으로 적용된다. 그 이유는, 약한 학습 모형의 학습 이후 판별하지 못한 데이터 포인트에 대해서 가중치를 부여하기 때문이다.

에이다 부스트의 핵심 아이디어는 분류하기 어려운 트레이닝 데이터에 가중치를 더 높이는 것이다. 즉, 이전에 잘못 분류된 트레이닝 데이터 포인트는 가중치가 증가해 오차율이 높아진다. 다음 약한 학습기는 이전에 증가한 오차를 낮추는 방향으로 학습하게 된다.
에이다 부스트는 일반적인 부스팅과는 다르게 약한 학습기를 훈련할 때 훈련 데이터셋 전체를 사용한다. 훈련 샘프은 반복할 때마다 가중치가 부여되며 이 앙상블은 이전 학습기의 실수한 부분을 학습하는 강력한 분류기를 만든다.

에이다 부스트 알고리즘을 활용해 암 여부를 예측하는 모형을 만든다.

In [2]:
# 데이터 불러오기
from sklearn import datasets  # 위스콘신 암 데이터를 사용하기 위해 import 한다.

raw_cancer = datasets.load_breast_cancer()  # 위스콘신 암 데이터를 불러온다.

In [3]:
# 피쳐/타겟 데이터 지정
X = raw_cancer.data  # 위스콘신 암 피쳐 데이터를 저장한다.
y = raw_cancer.target  # 위스콘신 암 타겟 데이터를 저장한다.

In [4]:
# 트레이닝/테스트 데이터 분할
from sklearn.model_selection import train_test_split  # 트레이닝/테스트 데이터 분할을 위해 import 한다.

X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)  # 트레이닝 데이터와 테스트 데이터로 분할한다.
X_tn.shape, X_te.shape

((426, 30), (143, 30))

In [5]:
# 데이터 표준화
from sklearn.preprocessing import StandardScaler  # 데이터 표준화를 위해 import 한다.

std_scale = StandardScaler()  # 표준화 스케일러 객체를 만든다.
# 표준화는 트레이닝 데이터를 기반으로 실행하므로 트레이닝 피쳐 데이터 X_tn을 표준화 스케일러에 적합시킨다.
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)  # 트레이닝 피쳐 데이터 X_tn을 표준화 한다.
X_te_std = std_scale.transform(X_te)  # 테스트 피쳐 데이터 X_te를 표준화 한다.

In [6]:
# 데이터 학습
# 분류 문제이므로 AdaBoostClassifier를 사용하지만 회귀 문제라면 AdaBoostRegressor를 사용한다.
from sklearn.ensemble import AdaBoostClassifier  # 에이다 부스트 알고리즘을 사용하기 위해 import 한다.

clf_ada = AdaBoostClassifier(random_state=0)  # 에이다 부스트 모델 객체를 만든다.
# 표준화된 피쳐 데이터 X_tn_std와 트레이닝 타겟 데이터 y_tn을 넣어서 랜덤 포레스트 알고리즘을 학습시킨다.
clf_ada.fit(X_tn_std, y_tn)

AdaBoostClassifier(random_state=0)

In [7]:
# 데이터 예측
pred_ada = clf_ada.predict(X_te_std)  # 표준화된 테스트 데이터 X_te_std로 예측한다.
pred_ada

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0])

In [8]:
# 정확도 평가
from sklearn.metrics import accuracy_score  # 모형 정확도를 평가하기 위해 import 한다.

# accuracy_score() 함수의 인수로 실제 타겟 데이터와 예측된 데이터를 넘겨 정확도를 평가한다.
accuracy = accuracy_score(y_te, pred_ada)
accuracy

0.9790209790209791

In [9]:
# 혼돈 행렬 확인
from sklearn.metrics import confusion_matrix  # 혼돈 행렬을 만들기 위해 import 한다.

# confusion_matrix() 함수에 실제 타겟 데이터와 예측된 데이터를 넘겨 혼돈 행렬을 만든다.
conf_matrix = confusion_matrix(y_te, pred_ada)
conf_matrix

array([[52,  1],
       [ 2, 88]])

In [10]:
# 분류 리포트 확인
from sklearn.metrics import classification_report  # 분류 리포트를 출력하기 위해 import 한다.

# classification_report() 함수에 실제 타겟 데이터와 예측된 데이터를 넘겨 분류 리포트를 만든다.
class_report = classification_report(y_te, pred_ada)
print(class_report)

              precision    recall  f1-score   support

           0       0.96      0.98      0.97        53
           1       0.99      0.98      0.98        90

    accuracy                           0.98       143
   macro avg       0.98      0.98      0.98       143
weighted avg       0.98      0.98      0.98       143



# 그래디언트 부스팅(Graient boosting)
그래디언트 부스팅은 부스팅의 한 종류로 그래디언트를 이용해 부스팅을 하는 방법이다. 그래디언트 부스팅은 비용 함수를 최적화 시킴으로써 학습 능력을 향상시키는 알고리즘이다.

그래디언트 부스팅 알고리즘을 활용해 암 여부를 예측하는 모형을 만든다.

In [11]:
# 데이터 학습
# 분류 문제이므로 GradientBoostingClassifier를 사용하지만 회귀 문제라면 GradientBoostingRegressor를 사용한다.
from sklearn.ensemble import GradientBoostingClassifier  # 그래디언트 부스트 알고리즘을 사용하기 위해 import 한다.

# 그래디언트 부스트 모델 객체를 만든다.
clf_gbt = GradientBoostingClassifier(max_depth=2, learning_rate=0.01, random_state=0)
# 표준화된 피쳐 데이터 X_tn_std와 트레이닝 타겟 데이터 y_tn을 넣어서 랜덤 포레스트 알고리즘을 학습시킨다.
clf_gbt.fit(X_tn_std, y_tn)

GradientBoostingClassifier(learning_rate=0.01, max_depth=2, random_state=0)

In [12]:
# 데이터 예측
pred_gboost = clf_gbt.predict(X_te_std)  # 표준화된 테스트 데이터 X_te_std로 예측한다.
pred_gboost

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0])

In [13]:
# 정확도 평가
from sklearn.metrics import accuracy_score  # 모형 정확도를 평가하기 위해 import 한다.

# accuracy_score() 함수의 인수로 실제 타겟 데이터와 예측된 데이터를 넘겨 정확도를 평가한다.
accuracy = accuracy_score(y_te, pred_gboost)
accuracy

0.965034965034965

In [14]:
# 혼돈 행렬 확인
from sklearn.metrics import confusion_matrix  # 혼돈 행렬을 만들기 위해 import 한다.

# confusion_matrix() 함수에 실제 타겟 데이터와 예측된 데이터를 넘겨 혼돈 행렬을 만든다.
conf_matrix = confusion_matrix(y_te, pred_gboost)
conf_matrix

array([[49,  4],
       [ 1, 89]])

In [15]:
# 분류 리포트 확인
from sklearn.metrics import classification_report  # 분류 리포트를 출력하기 위해 import 한다.

# classification_report() 함수에 실제 타겟 데이터와 예측된 데이터를 넘겨 분류 리포트를 만든다.
class_report = classification_report(y_te, pred_gboost)
print(class_report)

              precision    recall  f1-score   support

           0       0.98      0.92      0.95        53
           1       0.96      0.99      0.97        90

    accuracy                           0.97       143
   macro avg       0.97      0.96      0.96       143
weighted avg       0.97      0.97      0.96       143

